# K-Nearest Neighbors

Up until this point, linear regression has been the only type of machine learning model that we have used. There are many other classes of models that take a different approach to learning from data. In this chapter, we will use k-nearest neighbors (KNN) to model the relationship between housing features and the sale price. KNN is one of the simplest machine learning models.

## How KNN works

KNN works for both regression and classification supervised learning problems. Before performing KNN, you must choose `k`, the number of nearest neighbors for your model. We will only cover the regression case here. Like all supervised learning, you must begin with labeled training data.

Once you have your training data, you are set to make predictions. When you have a new observation that you'd like to use to make a prediction, find the distance between it and every single other observation in your training set. Select the `k` training observations with the least distance. The average target value of these `k` nearest neighbors will be the prediction.

### How do you define distance?

When using KNN, a distance metric must be defined. There are many distance metrics that have been developed, with Euclidean distance being the most common. Most people are taught the following formula for the Euclidean distance between two points $(x_{11}, x_{21})$ and $(x_{12}, x_{22})$

$$ d = \sqrt{(x_{11} - x_{12})^2 + (x_{21} - x_{22})^2}$$

With $p$ features this becomes:

$$ d = \sqrt{(x_{11} - x_{21})^2 + (x_{12} - x_{22})^2 + ... + (x_{p1} - x_{p2})^2}$$

## KNN in pandas

This algorithm is possible to implement using pandas due to its simplicity. Let's begin with just a single feature, `GrLivArea`.

In [ ]:
import pandas as pd
import numpy as np
housing = pd.read_csv('../data/housing_sample.csv')
housing.head()

A copy of the data is made to avoid a pandas warning.

In [ ]:
df = housing[['GrLivArea', 'SalePrice']].copy()
df.head()

Let's say we want to predict the price of a 1,500 square foot house using the 5 nearest neighbors. First, let's find the distance between each observation and 1,500 and assign this as a new column in our DataFrame. Because we are only in one dimension, the distance formula is simple and equal to the absolute value of the difference in values.

In [ ]:
df['distance'] = (df['GrLivArea'] - 1500).abs()
df.head()

### Use `nsmallest` to get the nearest neighbors

We can use the `nsmallest` method to get the 5 nearest neighbors. Three houses have an above ground living area equal to exactly 1,500 for a distance of 0. Two other houses are just one square foot more.

In [ ]:
df_neighbors = df.nsmallest(5, 'distance')
df_neighbors

### Average the sale price of these 5 nearest neighbors
The predicted value is the average target value of these 5 nearest neighbors.

In [ ]:
df_neighbors['SalePrice'].mean()

## KNN with multiple features

The algorithm works similarly when there are more features in the model. The only thing that changes is that the distance metric takes longer to compute as there are more dimensions. Let's do one more round of KNN by adding the variable `GarageArea` into the mix.

In [ ]:
df = housing[['GrLivArea', 'GarageArea', 'SalePrice']].copy()
df.head()

### Compute the distance
Let's predict the sale price for a home with above ground living area of 2,500 and a garage area of 800 garage area. First we compute the distance between this specific point and all other houses in the dataset. We add this distance as a column in our DataFrame.

In [ ]:
df['distance'] = np.sqrt((df['GrLivArea'] - 2500) ** 2 + (df['GarageArea'] - 800) ** 2)
df.head()

Again, we will use the `nsmallest` method to choose the 5 nearest neighbors.

In [ ]:
df_neighbors = df.nsmallest(5, 'distance')
df_neighbors

Once we have the nearest neighbors, we average the sale price to get the prediction.

In [ ]:
df_neighbors['SalePrice'].mean()

## Visualizing KNN

With just two features, we can create a scatter plot of all the points in the dataset along with the point we are trying to predict. This allows us to view which neighbors are likely to be nearest. Below, the black point represents the house we would like to make a prediction on.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
ax = df.plot(x='GrLivArea', y='GarageArea', kind='scatter', figsize=(14, 6))
ax.scatter([2500], [800], color='black', s=40, label='Predicted Value');

A function is defined below that will make the same scatter plot as above, but also find the given `k` nearest neighbors and plot them as red points. The predicted sale price is placed in the title.

In [ ]:
def plot_neighbors(ground_area, garage_area, k):
    df['distance'] = np.sqrt((df['GrLivArea'] - ground_area) ** 2 + (df['GarageArea'] - garage_area) ** 2)
    df_neighbors = df.nsmallest(k, 'distance')
    mean = df_neighbors['SalePrice'].mean()
    ax = df.plot(x='GrLivArea', y='GarageArea', kind='scatter', figsize=(14, 6))
    df_neighbors.plot(x='GrLivArea', y='GarageArea', kind='scatter', color='red', ax=ax, label=f'{k} neighbors')
    ax.scatter([ground_area], [garage_area], color='black', s=40, label='Predicted value')
    ax.set_title(f'{k} nearest neighbors of {ground_area} square feet and {garage_area} '
                 f'garage square feet predict sale price of \${mean:,.0f}')
    ax.legend()

Let's run this function to find the neighbors and get the predicted price.

In [ ]:
plot_neighbors(2500, 800, 5)

## Use Scikit-Learn
Of course, there is no need to use pandas as scikit-learn can perform KNN easily in just a few line of code. By default, scikit-learn uses 5 neighbors with Euclidean distance. Let's continue to use the same two features from above and assign our input and output data to `X` and `y`.

In [ ]:
X = housing[['GrLivArea', 'GarageArea']]
y = housing['SalePrice']

### Import, Instantiate, Fit

As always, we'll complete the three-step process to learn from data. The KNN model is named `KNeighborsRegressor` and found in the `neighbors` module. Make sure to not import `KNeighborsClassifier`, which is used for classification.

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
knr = KNeighborsRegressor(n_neighbors=5)
knr.fit(X, y)

### Verify that we get the same result as above

Let's use our new model to make a prediction on the same input data as we did above using pandas. We should get the same result.

In [ ]:
knr.predict([[2500, 800]])

## Measuring Performance of KNN

By default, scikit-learn uses $R^2$ for all regression learning models. Again, it provides a relative score showing how much better our model is versus guessing the mean. Any positive score shows improvement over the mean with a maximum score of 1 achieved when there is 0 error. We will soon learn how to properly evaluate machine learning models, but for now, we can just score the model with the training data.

In [ ]:
knr.score(X, y)

### Get (nearly) a perfect score with k=1

If you build a KNN model using just one neighbor and then score the model on the training data, you will get a perfect score (or very close to it). This is because the nearest neighbor will be itself. The only reason, you might not get exactly a perfect score (as is the case below) is because there might be multiple observations with the same exact values for each feature, but have a different target value. When two points are equidistant from the predicted point, scikit-learn chooses the one that comes first in the dataset to be the neighbor.

In [ ]:
knr1 = KNeighborsRegressor(n_neighbors=1)
knr1.fit(X, y)
knr1.score(X, y)

## Notes on KNN

### Curse of Dimensionality

As the number of dimensions grows, the "nearness" of the neighbors becomes more arbitrary. For example, if we have 20 features, our distance formula will be squaring the difference between 20 variables. Comparing the differences in distances between two points becomes very difficult in high dimensional space. If we were to use our own universe as an example, it would be like choosing between neighbors that are either 30 or 3,000 light years away. While the neighbor 30 light years away is 100 times closer than the next neighbor, it still isn't that close. There might be no close neighbors. This obstacle is often referred to as the **curse of dimensionality**.

### Nothing happens during the `fit` method

In scikit-learn, KNN is implemented as a **lazy** learner. When the `fit` method is called, no learning takes place.  The model simply stores the training data so that it can perform distance calculations when predicting. The learning (calculation of distance to find nearest neighbors) takes place during prediction.

### Prediction can take a long time

If the number of training examples is large, prediction can be time consuming. For each observation that you desire to predict, the distance from it to all other training observations must be calculated.

This is completely different than what happens with a linear regression. Predicting is very fast with linear regression, because you just need to multiply the coefficients by the feature values and sum up the result. Training a linear regression, on the other hand, is going to be slower.

### Using proportional distance

By default, scikit-learn weighs all of the neighbors equally when making a prediction. For instance, when the `k` nearest neighbors are found, a straight average is found for all `k` of these points regardless of the actual distance between it and the observation. Intuitively, it makes sense to give more weight to those neighbors that are closest. We can change this behavior such that a weighted average that is proportional to the distance is used by setting the `weights` parameter during instantiation to 'distance'. By default, it is 'uniform'.

Let's instantiate another KNN model so that it uses proportional distance weighting and then make a prediction on the same point from above. You'll notice that the predicted sale price has changed.

In [ ]:
knr = KNeighborsRegressor(n_neighbors=5, weights='distance')
knr.fit(X, y)
knr.predict([[2500, 800]])

## Distance calculation on features of different scale

When different features have vastly different scales, the distance might not return an accurate metric. In the housing dataset, we have features like `GrLivArea` which range in the thousands, while a feature like `OverallQual` ranges from 1 to 10. An extreme example can help make this issue clearer. 

Let's say we have three houses with the following values for `GrLivArea` and `OverallQual`. We are interested in finding whether house A is nearer to house B or C.

| House | GrLivArea | OverallQual |
|-------|-----------|----------|
| A     | 2,500     | 8        |
| B     | 2,400     | 1        |
| C     | 2,300     | 7        |


Using the distance formula, house B results in a far smaller distance to A than C. This is because the living area has a scale much greater than the overall quality. The difference in overall quality adds nearly nothing to the distance calculation.

For our distance measure to be more consistent, we need to scale the features so that they are all in similar ranges. There are a few strategies on how to do this transformation that will be introduced in later chapters.

## The fewer the neighbors the higher the variance

You are allowed to set `k` to be any number between 1 and the the total number of observations in the dataset. If we allow `k` to be the number of observations in this dataset (1,460 in this case), then every point will be a neighbor and the predicted value will be the mean of the entire dataset for every new observation. On the other end of the spectrum, if `k` is set to 1, then the predictions will vary substantially as they will depend on only a single neighbor. Let's verify that using `k` equal to `n`, the number of observations, leads to every predicted value as the mean. The first 10 predicted values are outputted.

In [ ]:
knr = KNeighborsRegressor(n_neighbors=len(X))
knr.fit(X, y)
y_pred = knr.predict(X).round()
y_pred[:10]

The higher the value of `k`, the more stable your prediction will be. This tradeoff between high variability and stability is known as the **bias-variance tradeoff** and will be discussed later.

## Exercises

### Exercise 1
<span  style="color:green; font-size:16px">Make a few more KNN models by varying the number of neighbors. Also, build models with different values of the `weights` parameter. Compute the score on each.</span>

### Exercise 2

<span  style="color:green; font-size:16px">Build a KNN model with just the features `GrLivArea` and `GarageArea`. Build 10 separate models all with values of `k` that range somewhere between 1 and 1000. For each model, make a prediction for every value in the training set. Find the variance of each of the 10 sets of predictions. Then make a line plot of the variances on the y-axis and `k` on the x-axis.</span>